In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

class GeoBrown:

    def time_step(self):
        # Brownian motion
        dW = np.random.normal(0, math.sqrt(self.dt))
        # stochastic process of the stock price model
        dS = self.r*self.current_S_T*self.dt + self.sigma*self.current_S_T*dW
        # updating the asset price
        self.S_T.append(self.current_S_T + dS)
        self.current_S_T = self.current_S_T + dS

    def __init__(self, S_T, r, dt, volatility):
        self.current_S_T = S_T
        self.S_T = []
        self.S_T.append(S_T)
        self.r = r
        self.dt = dt
        self.sigma = volatility

class MonteCarlo:
    
    def __init__(self, option ,strike, n_options, S_0, r, dt, volatility, tot_T):
        # generate the geometric paths 
        processes =[]
        for i in range(0, n_options):
            processes.append(GeoBrown(S_0,r,dt,volatility))
            
        # march in time in stochastic model for asset price calculation
        for process in processes:
            T_iter= tot_T
            while((T_iter-process.dt) > 0):
                T_iter = T_iter- process.dt
                process.time_step()
        
        # implementing the payoff functions for European and Asian call options
        payoffs = []
        if option == 'Euro':
            for process in processes:
                payoff =  process.S_T[len(process.S_T)-1] - strike
                fT = payoff if payoff > 0 else 0
                payoffs.append(fT)
                
        if option == 'Asian':
            for process in processes:
                payoff =  np.average(process.S_T) - strike
                fT = payoff  if payoff > 0 else 0
                payoffs.append(fT)
        
        # payoff function at time T
        self.V = np.average(payoffs)*math.exp(-r*tot_T)
        

In [ ]:
# outputs and plots for European call option

# input parameters for European call option
call_option = 'Euro'
strike = 100
N_geo_BM = 10000 
init_stock_price = 95
volatility = np.sqrt(0.1)
risk_free_rate = 0.02
payoff_time = 1
dt = 1/365  # marching time is everyday over time period

print('Monte Carlo European Call Price:')
print(MonteCarlo(call_option,strike, N_geo_BM, init_stock_price,risk_free_rate,dt,volatility,payoff_time).V)

# plotting the value of European call option vs initial stock price
value = []
for i in range(1,200):
    value.append(MonteCarlo(call_option,strike, N_geo_BM, i,risk_free_rate,dt,volatility,payoff_time).V)

plt.figure(0)    
plt.plot(value)
titl= call_option + ' call option'
plt.title(titl)
plt.xlabel("Initial stock price $S_{0}$")
plt.ylabel("Value of call option")
savefig=call_option + '_call_option.png'
plt.savefig(savefig)

In [ ]:
# outputs and plots for Asian call option

# input parameters for Asian call option
call_option = 'Asian'
strike = 100  
N_geo_BM = 10000 
init_stock_price = 95
volatility = np.sqrt(0.1)
risk_free_rate = 0.02
payoff_time = 1
dt = 1/4  # marching time is quarterly over time period

print('Monte Carlo Asian Call Price:')
print(MonteCarlo(call_option,strike, N_geo_BM, init_stock_price,risk_free_rate,dt,volatility,payoff_time).V)

# plotting the value of Asian call option vs initial stock price
value = []
for i in range(1,200):
    value.append(MonteCarlo(call_option,strike, N_geo_BM, i,risk_free_rate,dt,volatility,payoff_time).V)
    
plt.figure(1)
plt.plot(value)
titl= call_option + ' call option'
plt.title(titl)
plt.xlabel("Initial stock price $S_{0}$")
plt.ylabel("Value of call option")
savefig=call_option + '_call_option.png'
plt.savefig(savefig)

In [ ]:
### sensitivities: risk financial derivatives
# finite difference method (FDM) is used to descretize the sensitivities 

class Sensitivities:
    
    def __init__(self, call_option, strike, n_options, S_0, r, dt, volatility, tot_T):
        
        # FDM with central difference approach for delta
        S0_1 = S_0*0.95
        S0_2 = S_0*1.05
        V1 = MonteCarlo(call_option, strike, n_options, S0_1,r,dt,volatility,tot_T).V
        V2 = MonteCarlo(call_option, strike, n_options, S0_2,r,dt,volatility,tot_T).V
        self.delta = (V2-V1)/(S0_2-S0_1)
        
        # FDM with central difference approach for gamma
        V3 = MonteCarlo(call_option, strike, n_options, S_0,r,dt,volatility,tot_T).V
        self.gamma = (V2-2*V3+V1)/((S0_2-S0_1)**2.)
        
        # FDM with central difference approach for vega
        vol_1 = volatility*0.95
        vol_2 = volatility*1.05
        V1 = MonteCarlo(call_option, strike, n_options, S_0,r,dt,vol_1,tot_T).V
        V2 = MonteCarlo(call_option, strike, n_options, S_0,r,dt,vol_2,tot_T).V
        self.vega = (V2-V1)/(vol_2-vol_1)
        
        # FDM with central difference approach for cross gamma
        V1 = MonteCarlo(call_option, strike, n_options, S0_1,r,dt,vol_1,tot_T).V
        V2 = MonteCarlo(call_option, strike, n_options, S0_1,r,dt,vol_2,tot_T).V
        
        V3 = MonteCarlo(call_option, strike, n_options, S0_2,r,dt,vol_1,tot_T).V
        V4 = MonteCarlo(call_option, strike, n_options, S0_2,r,dt,vol_2,tot_T).V        
        self.cgamma = ((V4-V3)-(V2-V1))/((S0_2-S0_1)*(vol_2-vol_1))
        


In [ ]:
# calculation of sensitivities for European call option

call_option = 'Euro'
strike = 100  
N_geo_BM = 1000000
init_stock_price = 95
volatility = np.sqrt(0.1)
risk_free_rate = 0.02
payoff_time = 1
dt = 1/4  

Sensit = Sensitivities(call_option,strike, N_geo_BM, init_stock_price,risk_free_rate,dt,volatility,payoff_time)


In [ ]:
print('Delta value for',call_option,'call option is:',Sensit.delta)
print('Vega value for',call_option,'call option is:',Sensit.vega)
print('Gamma value for',call_option,'call option is:',Sensit.gamma)
print('Cross gamma value for',call_option,'call option is:',Sensit.cgamma)

In [ ]:
# plotting the value of European call option vs initial stock price
max_S = 200  #maximum stock price for plotting the sensivities
i = 0
skip = 10  # skiping jump between the stock prices
delta=[]
vega=[]
gamma=[]
cgamma=[]
iterate=[]
while i < max_S:
    i += skip
    iterate.append(i)
    sensit=Sensitivities(call_option,strike, N_geo_BM, i,risk_free_rate,dt,volatility,payoff_time)
    delta.append(sensit.delta)
    vega.append(sensit.vega)
    gamma.append(sensit.gamma)
    cgamma.append(sensit.cgamma)

In [ ]:
# plotting and saving the plots for Greeks
fignum=2
plt.figure(fignum)
plt.plot(iterate,delta)
titl= call_option + ' call option'
plt.title(titl)
plt.xlabel("Initial stock price $S_{0}$")
plt.ylabel("Delta")
savefig=call_option + '_delta.png'
plt.savefig(savefig)

fignum = fignum+1
plt.figure(fignum)
plt.plot(iterate,vega)
titl= call_option + ' call option'
plt.title(titl)
plt.xlabel("Initial stock price $S_{0}$")
plt.ylabel("Vega")
savefig=call_option + '_vega.png'
plt.savefig(savefig)

fignum = fignum+1
plt.figure(fignum)
plt.plot(iterate,gamma)
titl= call_option + ' call option'
plt.title(titl)
plt.xlabel("Initial stock price $S_{0}$")
plt.ylabel("Gamma")
savefig=call_option + '_gamma.png'
plt.savefig(savefig)

fignum = fignum+1
plt.figure(fignum)
plt.plot(iterate,cgamma)
titl= call_option + ' call option'
plt.title(titl)
plt.xlabel("Initial stock price $S_{0}$")
plt.ylabel("Cross gamma")
savefig=call_option + '_cgamma.png'
plt.savefig(savefig)

In [ ]:
#calculation of sensitivities for Asian call option

call_option = 'Asian'
strike = 100  
N_geo_BM = 1000000
init_stock_price = 95
volatility = np.sqrt(0.1)
risk_free_rate = 0.02
payoff_time = 1
dt = 1/4  

Sensit = Sensitivities(call_option,strike, N_geo_BM, init_stock_price,risk_free_rate,dt,volatility,payoff_time)


In [ ]:
print('Delta value for',call_option,'call option is:',Sensit.delta)
print('Vega value for',call_option,'call option is:',Sensit.vega)
print('Gamma value for',call_option,'call option is:',Sensit.gamma)
print('Cross gamma value for',call_option,'call option is:',Sensit.cgamma)

In [ ]:
# plotting the value Asian call option vs initial stock price
max_S = 200  #maximum stock price for plotting the sensivities
i = 0
skip = 10  # skiping jump between the stock prices
delta=[]
vega=[]
gamma=[]
cgamma=[]
iterate=[]
while i < max_S:
    i += skip
    iterate.append(i)
    sensit=Sensitivities(call_option,strike, N_geo_BM, i,risk_free_rate,dt,volatility,payoff_time)
    delta.append(sensit.delta)
    vega.append(sensit.vega)
    gamma.append(sensit.gamma)
    cgamma.append(sensit.cgamma)

In [ ]:
# plotting and saving the plots for Greeks
call_option = 'Asian'
fignum = fignum+1
plt.figure(fignum)
plt.plot(iterate,delta)
titl= call_option + ' call option'
plt.title(titl)
plt.xlabel("Initial stock price $S_{0}$")
plt.ylabel("Delta")
savefig=call_option + '_delta.png'
plt.savefig(savefig)

fignum = fignum+1
plt.figure(fignum)
plt.plot(iterate,vega)
titl= call_option + ' call option'
plt.title(titl)
plt.xlabel("Initial stock price $S_{0}$")
plt.ylabel("Vega")
savefig=call_option + '_vega.png'
plt.savefig(savefig)

fignum = fignum+1
plt.figure(fignum)
plt.plot(iterate,gamma)
titl= call_option + ' call option'
plt.title(titl)
plt.xlabel("Initial stock price $S_{0}$")
plt.ylabel("Gamma")
savefig=call_option + '_gamma.png'
plt.savefig(savefig)

fignum = fignum+1
plt.figure(fignum)
plt.plot(iterate,cgamma)
titl= call_option + ' call option'
plt.title(titl)
plt.xlabel("Initial stock price $S_{0}$")
plt.ylabel("Cross gamma")
savefig=call_option + '_cgamma.png'
plt.savefig(savefig)